In [48]:
import pandas as pd
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [49]:
review_df = pd.read_csv("amazon-customer-reviews.csv")
review_df.head()

,userID,review_date,productID,review_helpful,review_text,review_summary
0,A1BWD4UA5QTBOI,2013-10-28,B004BQKQ8A,"[0, 0]","['card', 'work', 'expected', 'fast', 'great', ...","['bought', 'older', 'computer']"
1,A27JB8ALUWAECO,2014-01-16,B005O74J7O,"[0, 0]","['well', 'case', 'go', 'one', 'look', 'good', ...",[]
2,A32B6A532454UD,2012-06-12,B005P9CATU,"[10, 10]","['using', 'app', 'power', 'connector', 'year',...","['connector', 'standard', 'radio', 'us', '30',..."
3,A225M3GAYN01IG,2011-04-10,B0023APPCI,"[1, 1]","['model', 'use', 'led', 'hd', 'tv', 'bluray', ...","['great', 'sound', 'easy', 'set', 'look', 'ama..."
4,AX2F72W2O86WI,2012-06-07,B0055LGG3Y,"[3, 4]","['system', 'using', 'memory', 'system', 'purch...","['great', 'memory']"


In [50]:
# Function to perform semantic analysis
def analyze_semantics(text):
    # Tokenize the text
    words = word_tokenize(text)

    # Calculate sentiment using TextBlob
    analysis = TextBlob(" ".join(words))
    sentiment = analysis.sentiment.polarity

    return sentiment

# Apply semantic analysis to the "review_summary" column
review_df['semantic_value_text'] = review_df['review_text'].apply(analyze_semantics)
review_df['semantic_value_summary'] = review_df['review_summary'].apply(analyze_semantics)



In [51]:
review_df.head()

,userID,review_date,productID,review_helpful,review_text,review_summary,semantic_value_text,semantic_value_summary
0,A1BWD4UA5QTBOI,2013-10-28,B004BQKQ8A,"[0, 0]","['card', 'work', 'expected', 'fast', 'great', ...","['bought', 'older', 'computer']",0.330000,0.166667
1,A27JB8ALUWAECO,2014-01-16,B005O74J7O,"[0, 0]","['well', 'case', 'go', 'one', 'look', 'good', ...",[],0.090278,0.000000
2,A32B6A532454UD,2012-06-12,B005P9CATU,"[10, 10]","['using', 'app', 'power', 'connector', 'year',...","['connector', 'standard', 'radio', 'us', '30',...",0.111329,0.000000
3,A225M3GAYN01IG,2011-04-10,B0023APPCI,"[1, 1]","['model', 'use', 'led', 'hd', 'tv', 'bluray', ...","['great', 'sound', 'easy', 'set', 'look', 'ama...",0.420000,0.558333
4,AX2F72W2O86WI,2012-06-07,B0055LGG3Y,"[3, 4]","['system', 'using', 'memory', 'system', 'purch...","['great', 'memory']",0.000000,0.800000
